# Narrative Gravity Wells - Exploratory Data AnalysisStudy: iditi_validation_study

In [ ]:
import pandas as pdimport numpy as npfrom scipy import statsimport plotly.express as pximport plotly.graph_objects as gofrom plotly.subplots import make_subplotsimport warningswarnings.filterwarnings('ignore')# Import centralized visualization systemfrom narrative_gravity.visualization import create_visualization_engineprint("📊 Narrative Gravity Wells Analysis")print("=" * 50)

In [ ]:
# Load datasetprint("Loading dataset...")data = pd.read_feather('../data/iditi_validation_study.feather')print(f"✅ Loaded {len(data)} observations with {len(data.columns)} variables")print(f"Date range: {data['exp_date'].min()} to {data['exp_date'].max()}")print(f"Frameworks: {data['framework'].unique()}")print(f"LLM Models: {data['llm_model'].unique()}")

## Dataset Overview

In [ ]:
# Dataset structureprint("\nDataset Info:")print(data.info())# Basic statisticsprint("\nDescriptive Statistics:")print(data.describe())# Missing data analysisprint("\nMissing Data Analysis:")missing = data.isnull().sum()missing_pct = (missing / len(data)) * 100missing_df = pd.DataFrame({    'Missing Count': missing,    'Missing %': missing_pct}).sort_values('Missing %', ascending=False)print(missing_df[missing_df['Missing Count'] > 0])

## Reliability Analysis

In [ ]:
# Reliability by frameworkprint("\nReliability Analysis by Framework:")reliability_stats = data.groupby('framework')['cv'].agg([    'count', 'mean', 'std', 'min', 'max']).round(4)print(reliability_stats)# Reliability target analysistarget_cv = 0.20reliable_analyses = data[data['cv'] <= target_cv]reliability_rate = len(reliable_analyses) / len(data.dropna(subset=['cv'])) * 100print(f"\nReliability Rate (CV ≤ {target_cv}): {reliability_rate:.1f}%")# Framework reliability comparisonframework_reliability = data.groupby('framework').apply(    lambda x: (x['cv'] <= target_cv).mean() * 100).round(1)print("\nReliability Rate by Framework:")print(framework_reliability.sort_values(ascending=False))

## Framework Performance

In [ ]:
# Framework performance comparisonprint("\nFramework Performance Analysis:")framework_stats = data.groupby('framework').agg({    'cv': ['mean', 'std', 'count'],    'icc': ['mean', 'std'],    'cost': ['mean', 'sum'],    'process_time_sec': ['mean', 'sum']}).round(4)# Flatten column namesframework_stats.columns = ['_'.join(col).strip() for col in framework_stats.columns]print(framework_stats)# Statistical significance testingfrom scipy import statsframeworks = data['framework'].unique()cv_by_framework = [data[data['framework'] == f]['cv'].dropna() for f in frameworks]if len(cv_by_framework) > 1 and all(len(group) > 1 for group in cv_by_framework):    f_stat, p_value = stats.f_oneway(*cv_by_framework)    print(f"\nFramework Effect on Reliability:")    print(f"F-statistic: {f_stat:.4f}, p-value: {p_value:.4f}")        if p_value < 0.05:        print("✅ Significant framework effect detected")    else:        print("❌ No significant framework effect")

## Visualization

In [ ]:
# Professional visualization suite using centralized systemprint("\n📊 Creating professional visualizations...")# Initialize visualization engine with academic themeengine = create_visualization_engine(theme='academic')# 1. Reliability by Framework Analysisif 'cv' in data.columns and data['cv'].notna().any():    reliability_fig = px.box(        data,         x='framework',         y='cv',        title='Reliability by Framework (Lower CV = Better)',        labels={'cv': 'Coefficient of Variation', 'framework': 'Framework'}    )    reliability_fig.add_hline(        y=0.20,         line_dash="dash",         line_color="red",        annotation_text="Target Threshold (0.20)"    )    reliability_fig.update_layout(title_font_size=18, xaxis_tickangle=45)    reliability_fig.write_html('reliability_by_framework.html')    reliability_fig.write_image('reliability_by_framework.png', width=1200, height=800, scale=2)    print("✅ Reliability analysis: reliability_by_framework.html/.png")# 2. Model Performance Comparisonif 'llm_model' in data.columns and 'cv' in data.columns:    model_fig = px.violin(        data,         x='llm_model',         y='cv',        title='Model Performance Comparison',        labels={'cv': 'Coefficient of Variation', 'llm_model': 'LLM Model'}    )    model_fig.update_layout(title_font_size=18, xaxis_tickangle=45)    model_fig.write_html('model_performance.html')    model_fig.write_image('model_performance.png', width=1200, height=800, scale=2)    print("✅ Model performance: model_performance.html/.png")# 3. Cost vs Processing Time Analysisif 'cost' in data.columns and 'process_time_sec' in data.columns:    cost_fig = px.scatter(        data,         x='process_time_sec',         y='cost',         color='llm_model',        title='Cost vs Processing Time Analysis',        labels={            'process_time_sec': 'Processing Time (seconds)',             'cost': 'Cost (USD)',            'llm_model': 'LLM Model'        }    )    cost_fig.update_layout(title_font_size=18)    cost_fig.write_html('cost_analysis.html')    cost_fig.write_image('cost_analysis.png', width=1200, height=800, scale=2)    print("✅ Cost analysis: cost_analysis.html/.png")# 4. Framework Timeline Analysisif 'exp_date' in data.columns:    data_time = data.copy()    data_time['month'] = pd.to_datetime(data_time['exp_date']).dt.to_period('M').astype(str)    framework_timeline = data_time.groupby(['month', 'framework']).size().reset_index(name='count')        timeline_fig = px.line(        framework_timeline,         x='month',         y='count',         color='framework',        title='Framework Usage Timeline',        labels={'count': 'Number of Analyses', 'month': 'Month'}    )    timeline_fig.update_layout(title_font_size=18, xaxis_tickangle=45)    timeline_fig.write_html('framework_timeline.html')    timeline_fig.write_image('framework_timeline.png', width=1200, height=800, scale=2)    print("✅ Timeline analysis: framework_timeline.html/.png")# 5. Well Scores Distributionwell_columns = [col for col in data.columns if col.startswith('well_')]if well_columns:    well_data = data[well_columns].melt(var_name='well', value_name='score')    well_fig = px.box(        well_data,         x='well',         y='score',        title='Well Scores Distribution',        labels={'score': 'Score', 'well': 'Narrative Well'}    )    well_fig.update_layout(title_font_size=18, xaxis_tickangle=45)    well_fig.write_html('well_scores_distribution.html')    well_fig.write_image('well_scores_distribution.png', width=1200, height=800, scale=2)    print("✅ Well scores: well_scores_distribution.html/.png")# 6. Reliability Improvement Over Timeif 'exp_date' in data.columns and 'cv' in data.columns:    data_time = data.copy()    data_time['week'] = pd.to_datetime(data_time['exp_date']).dt.to_period('W').astype(str)    weekly_cv = data_time.groupby('week')['cv'].mean().reset_index()        improvement_fig = px.line(        weekly_cv,         x='week',         y='cv',        title='Reliability Improvement Over Time',        labels={'cv': 'Mean Coefficient of Variation', 'week': 'Week'}    )    improvement_fig.add_hline(        y=0.20,         line_dash="dash",         line_color="red",        annotation_text="Target Threshold"    )    improvement_fig.update_layout(title_font_size=18, xaxis_tickangle=45)    improvement_fig.write_html('reliability_improvement.html')    improvement_fig.write_image('reliability_improvement.png', width=1200, height=800, scale=2)    print("✅ Improvement tracking: reliability_improvement.html/.png")# Create comprehensive dashboard with all visualizationsprint("\n📊 Creating comprehensive dashboard...")dashboard_analyses = []# Add narrative gravity visualization if well data existsif well_columns and len(well_columns) >= 3:    # Extract sample well definitions for visualization    wells = {}    for i, well_col in enumerate(well_columns[:8]):  # Limit to 8 wells for clarity        well_name = well_col.replace('well_', '').replace('_', ' ').title()        angle = (i * 360 / len(well_columns[:8])) % 360        wells[well_name] = {            'angle': angle,            'type': 'integrative' if i % 2 == 0 else 'disintegrative',            'weight': 1.0        }        # Get latest analysis scores    if len(data) > 0:        latest_scores = {}        for well_col in well_columns[:8]:            well_name = well_col.replace('well_', '').replace('_', ' ').title()            latest_scores[well_name] = data[well_col].dropna().iloc[-1] if not data[well_col].dropna().empty else 0.5                dashboard_analyses.append({            'title': 'Latest Analysis Results',            'wells': wells,            'scores': latest_scores        })if dashboard_analyses:    dashboard_fig = engine.create_dashboard(        analyses=dashboard_analyses,        title='Narrative Gravity Analysis Dashboard',        include_summary=True    )    dashboard_fig.write_html('narrative_gravity_dashboard.html')    dashboard_fig.write_image('narrative_gravity_dashboard.png', width=1800, height=1200, scale=2)    print("✅ Dashboard: narrative_gravity_dashboard.html/.png")print("\n✅ Professional visualization suite completed!")print("📊 Interactive HTML files created for detailed exploration")print("📊 High-resolution PNG files created for publications")print("🎯 All visualizations use consistent academic theming")